# Random Forest

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from statistics import mean 

In [2]:
# Create Random Forest Classifier
rf = RandomForestClassifier(n_estimators=1024)

# Create parameters
param = {
    'max_features': [1, 2, 4, 6, 8, 12, 16, 20]
}

In [3]:
# Returns models with the best hyperparameters for random forest
def best_models(acc, roc, f1):
    acc_model = RandomForestClassifier(n_estimators=1024, max_features=acc['max_features'])
    roc_model = RandomForestClassifier(n_estimators=1024, max_features=roc['max_features'])
    f1_model = RandomForestClassifier(n_estimators=1024, max_features=f1['max_features'])
    
    return acc_model, roc_model, f1_model

In [4]:
TRIALS = 5 # num of trials to run

# *Adult Dataset*

In [5]:
# Import adult data set
adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
adult.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']

# Change income to binary classification -- actual target value
def income_to_binary(income):
    target = 0
    if income == " >50K":
        target = 1
    else:
        target = -1
    return target

# Apply function to income column
adult['income'] = adult['income'].astype('str')
adult['income'] = adult['income'].apply(income_to_binary)

adult

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,-1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,-1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,-1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,-1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,-1
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,-1
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,-1


In [6]:
# Data Preprocessing

# Separate x-values from target values
X = adult.drop(['income'], axis=1)
Y = adult['income']

# Encoder/scaler
ohe = OneHotEncoder()
scaler = StandardScaler()

# Columns to one-hot encode or scale
encode = ['workclass','education','marital-status','occupation','relationship','race','sex','native-country']
scale = ['age','fnlwgt','capital-gain','capital-loss','hours-per-week']

# Encode and scale the columns for training data and testing data
encoded_data = ohe.fit_transform(X[encode]).toarray() 
scaled_data  = pd.DataFrame( scaler.fit_transform(X[scale]), columns=scale, index=X[scale].index )

# Concatenate processed data
for index, category in enumerate( np.concatenate(ohe.categories_) ):
    scaled_data[category] = encoded_data[:,index]

X = scaled_data

X

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,?,Federal-gov,Local-gov,Never-worked,Private,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,0.030671,-1.063611,0.148453,-0.21666,-0.035429,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.837109,-1.008707,-0.145920,-0.21666,-2.222153,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.042642,0.245079,-0.145920,-0.21666,-0.035429,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.057047,0.425801,-0.145920,-0.21666,-0.035429,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.775768,1.408176,-0.145920,-0.21666,-0.035429,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.849080,0.639741,-0.145920,-0.21666,-0.197409,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32557,0.103983,-0.335433,-0.145920,-0.21666,-0.035429,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32558,1.423610,-0.358777,-0.145920,-0.21666,-0.035429,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32559,-1.215643,0.110960,-0.145920,-0.21666,-1.655225,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
# Initialize lists to store performance scores
train_acc_scores = []
test_acc_scores = []
train_roc_scores = []
test_roc_scores = []
train_f1_scores = []
test_f1_scores = []

counter =  1 # for random state

for i in range(TRIALS):
    # Gets 5000 random samples
    X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=5000)
    
    counter += 1
    
    # Create grid search
    clf = GridSearchCV(rf, param, cv=StratifiedKFold(n_splits=5),
                   scoring=['accuracy','roc_auc','f1'], refit=False, verbose=2, n_jobs=-1)
    
    # Fit grid search
    best_model = clf.fit(X_train, y_train)
    
    # Best accuracy
    acc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ]
    # Best roc_auc
    roc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc']) ]
    # Best f1
    f1 = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1']) ]
    
    # Model Training
    # Best accuracy model, Best roc auc model, Best f1 model
    acc_model, roc_model, f1_model = best_models(acc, roc, f1)

    # Model fitting on training data
    acc_model.fit(X_train, y_train)
    roc_model.fit(X_train, y_train)
    f1_model.fit(X_train, y_train)
    
    # Model predicting
    # Accuracy model
    acc_train_pred = acc_model.predict(X_train)
    acc_test_pred = acc_model.predict(X_test)

    # Roc Auc model
    roc_train_pred = roc_model.predict_proba(X_train)[:,1]
    roc_test_pred = roc_model.predict_proba(X_test)[:,1]

    # f1 model
    f1_train_pred = f1_model.predict(X_train)
    f1_test_pred = f1_model.predict(X_test)
    
    # Appending score performances to train/test array
    # Best accuracy model scores
    train_acc_scores.append( accuracy_score(y_train,acc_train_pred) )
    test_acc_scores.append( accuracy_score(y_test,acc_test_pred) )
    
    # Best roc auc model scores
    train_roc_scores.append( roc_auc_score(y_train,roc_train_pred) )
    test_roc_scores.append( roc_auc_score(y_test,roc_test_pred) )
    
    # Best f1 model scores
    train_f1_scores.append( f1_score(y_train,f1_train_pred) )
    test_f1_scores.append( f1_score(y_test,acc_test_pred) )

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   57.2s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   57.8s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   53.0s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   53.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   52.3s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   52.8s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   52.8s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   53.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   51.4s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   51.7s finished


In [8]:
# Saves scores to a DataFrame and prints
RF_ADULT_TRAIN = pd.DataFrame([train_acc_scores, train_roc_scores, train_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_ADULT_TRAIN.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_ADULT_TRAIN,'\n')

RF_ADULT_TEST = pd.DataFrame([test_acc_scores, test_roc_scores, test_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_ADULT_TEST.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_ADULT_TEST)

     TRIAL 1  TRIAL 2  TRIAL 3  TRIAL 4  TRIAL 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0 

      TRIAL 1   TRIAL 2   TRIAL 3   TRIAL 4   TRIAL 5
ACC  0.849897  0.851638  0.851021  0.853706  0.850767
ROC  0.900433  0.903124  0.900143  0.905685  0.903196
FSC  0.658973  0.659222  0.661221  0.667765  0.669187


#  *Letter Recognition - A-M / N-Z Dataset*

In [9]:
# Import letters dataset
letters = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', header=None)
letters.columns = ['capital letter','x-box horizontal position of box','y-box vertical position of box',
                   'width of box','height of box','onpix total # on pixels','x-bar mean x of on pixels in box',
                   'y-bar mean y of on pixels in box','x2bar mean x variance', 'y2bar mean y variance',
                   'xybar mean x y correlation','x2ybr mean of x * x * y','xy2br mean of x * y * y',
                   'x-ege mean edge count left to right','xegvy correlation of x-ege with y',
                   'y-ege mean edge count bottom to top','yegvx correlation of y-ege with x']

# Change values in 'capital letter' consisting of 'A-M' to 1, otherwise 0
letters['capital letter'] = letters['capital letter'].str.contains('[A-M]') # True if capital letter is in A-M, False otherwise
letters['capital letter'] = letters['capital letter'].astype(int) # Converts True/False to 1/0

# Changes value from 0 to -1
for index, value in letters['capital letter'].iteritems():
    if value == 0:
        letters.at[index,'capital letter'] = -1

letters

,capital letter,x-box horizontal position of box,y-box vertical position of box,width of box,height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,-1,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,1,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,-1,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,1,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,1,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,-1,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,-1,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [10]:
# Data Preprocessing

# Separate x-values from target values
X = letters.drop(['capital letter'], axis=1)
Y = letters['capital letter']

# Encoder/scaler
scaler = StandardScaler()

# Columns to scale
scale = ['x-box horizontal position of box','y-box vertical position of box',
         'width of box','height of box','onpix total # on pixels','x-bar mean x of on pixels in box',
         'y-bar mean y of on pixels in box','x2bar mean x variance', 'y2bar mean y variance',
         'xybar mean x y correlation','x2ybr mean of x * x * y','xy2br mean of x * y * y',
         'x-ege mean edge count left to right','xegvy correlation of x-ege with y',
         'y-ege mean edge count bottom to top','yegvx correlation of y-ege with x']

# Scale the columns for training data and testing data
scaled_data  = pd.DataFrame( scaler.fit_transform(X[scale]), columns=scale, index=X[scale].index )

X = scaled_data

X

,x-box horizontal position of box,y-box vertical position of box,width of box,height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,-1.057698,0.291877,-1.053277,-0.164704,-1.144013,0.544130,2.365097,-1.714360,0.344994,-0.917071,1.347774,0.034125,-1.305948,-0.219082,-1.438153,0.122911
1,0.510385,1.502358,-1.053277,0.719730,-0.687476,1.531305,-1.075326,0.137561,-0.495072,1.895968,-1.312807,0.514764,-0.448492,-0.219082,0.120081,1.359441
2,-0.012309,1.199738,0.435910,1.161947,1.138672,1.531305,-0.645273,-0.973591,0.344994,0.690380,-1.312807,-0.446513,-0.019764,-0.865626,-0.269477,0.741176
3,1.555774,1.199738,0.435910,0.277513,-0.230939,-0.936631,0.644886,-0.232823,0.344994,-1.720796,-0.932724,0.995402,1.266419,1.074008,-0.659036,0.122911
4,-1.057698,-1.826464,-1.053277,-1.933571,-1.144013,0.544130,-0.645273,0.507945,0.344994,-0.917071,-0.552641,0.514764,-0.877220,-0.865626,0.509640,1.359441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-1.057698,-1.523844,-1.053277,-1.049137,-0.687476,0.050543,-0.215220,0.878329,0.344994,-0.917071,-0.172558,-1.888428,-0.448492,-0.219082,-0.269477,-0.495354
19996,1.555774,0.897117,1.428701,1.161947,0.225598,-1.430218,0.214833,0.507945,1.605094,1.494105,0.967691,2.437316,-0.448492,0.427463,-0.269477,-0.495354
19997,1.033079,0.594497,0.435910,0.719730,0.682135,-0.443044,1.504991,-0.603207,0.765028,1.092242,0.967691,-1.407789,-0.448492,2.367097,-0.659036,-2.350149
19998,-1.057698,-1.221224,-0.556881,-1.491354,-1.144013,0.544130,-0.215220,-0.973591,0.344994,0.690380,-0.172558,0.034125,-0.877220,0.427463,0.509640,0.122911


In [11]:
# Initialize lists to store performance scores
train_acc_scores = []
test_acc_scores = []
train_roc_scores = []
test_roc_scores = []
train_f1_scores = []
test_f1_scores = []

counter =  1 # for random state

for i in range(TRIALS):
    # Gets 5000 random samples
    X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=5000)
    
    counter += 1
    
    # Create grid search
    clf = GridSearchCV(rf, param, cv=StratifiedKFold(n_splits=5),
                   scoring=['accuracy','roc_auc','f1'], refit=False, verbose=2, n_jobs=-1)
    
    # Fit grid search
    best_model = clf.fit(X_train, y_train)
    
    # Best accuracy
    acc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ]
    # Best roc_auc
    roc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc']) ]
    # Best f1
    f1 = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1']) ]
    
    # Model Training
    # Best accuracy model, Best roc auc model, Best f1 model
    acc_model, roc_model, f1_model = best_models(acc, roc, f1)

    # Model fitting on training data
    acc_model.fit(X_train, y_train)
    roc_model.fit(X_train, y_train)
    f1_model.fit(X_train, y_train)
    
    # Model predicting
    # Accuracy model
    acc_train_pred = acc_model.predict(X_train)
    acc_test_pred = acc_model.predict(X_test)

    # Roc Auc model
    roc_train_pred = roc_model.predict_proba(X_train)[:,1]
    roc_test_pred = roc_model.predict_proba(X_test)[:,1]

    # f1 model
    f1_train_pred = f1_model.predict(X_train)
    f1_test_pred = f1_model.predict(X_test)
    
    # Appending score performances to train/test array
    # Best accuracy model scores
    train_acc_scores.append( accuracy_score(y_train,acc_train_pred) )
    test_acc_scores.append( accuracy_score(y_test,acc_test_pred) )
    
    # Best roc auc model scores
    train_roc_scores.append( roc_auc_score(y_train,roc_train_pred) )
    test_roc_scores.append( roc_auc_score(y_test,roc_test_pred) )
    
    # Best f1 model scores
    train_f1_scores.append( f1_score(y_train,f1_train_pred) )
    test_f1_scores.append( f1_score(y_test,acc_test_pred) )

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   40.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   41.1s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   45.1s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   46.0s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   43.3s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   44.2s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   44.0s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   44.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   42.0s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   42.6s finished


In [12]:
# Saves scores to a DataFrame and prints
RF_LETTERAM_TRAIN = pd.DataFrame([train_acc_scores, train_roc_scores, train_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_LETTERAM_TRAIN.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_LETTERAM_TRAIN,'\n')

RF_LETTERAM_TEST = pd.DataFrame([test_acc_scores, test_roc_scores, test_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_LETTERAM_TEST.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_LETTERAM_TEST)

     TRIAL 1  TRIAL 2  TRIAL 3  TRIAL 4  TRIAL 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0 

      TRIAL 1   TRIAL 2   TRIAL 3   TRIAL 4   TRIAL 5
ACC  0.946867  0.945800  0.951000  0.944800  0.941467
ROC  0.990355  0.991099  0.991418  0.990945  0.989088
FSC  0.946297  0.945056  0.950964  0.944593  0.940812


#  *Letter Recognition - O / A-N + P-Z Dataset*

In [13]:
# Import letters dataset
letters = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', header=None)
letters.columns = ['capital letter','x-box horizontal position of box','y-box vertical position of box',
                   'width of box','height of box','onpix total # on pixels','x-bar mean x of on pixels in box',
                   'y-bar mean y of on pixels in box','x2bar mean x variance', 'y2bar mean y variance',
                   'xybar mean x y correlation','x2ybr mean of x * x * y','xy2br mean of x * y * y',
                   'x-ege mean edge count left to right','xegvy correlation of x-ege with y',
                   'y-ege mean edge count bottom to top','yegvx correlation of y-ege with x']

# Change values in 'capital letter' consisting of 'O' to 1, otherwise 0
letters['capital letter'] = letters['capital letter'].str.contains('O') # True if capital letter is in O, False otherwise
letters['capital letter'] = letters['capital letter'].astype(int) # Converts True/False to 1/0

# Changes value from 0 to -1
for index, value in letters['capital letter'].iteritems():
    if value == 0:
        letters.at[index,'capital letter'] = -1

letters

,capital letter,x-box horizontal position of box,y-box vertical position of box,width of box,height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,-1,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,-1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,-1,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,-1,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,-1,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-1,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,-1,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,-1,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,-1,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [14]:
# Data Preprocessing

# Separate x-values from target values
X = letters.drop(['capital letter'], axis=1)
Y = letters['capital letter']

# Encoder/scaler
scaler = StandardScaler()

# Columns to scale
scale = ['x-box horizontal position of box','y-box vertical position of box',
         'width of box','height of box','onpix total # on pixels','x-bar mean x of on pixels in box',
         'y-bar mean y of on pixels in box','x2bar mean x variance', 'y2bar mean y variance',
         'xybar mean x y correlation','x2ybr mean of x * x * y','xy2br mean of x * y * y',
         'x-ege mean edge count left to right','xegvy correlation of x-ege with y',
         'y-ege mean edge count bottom to top','yegvx correlation of y-ege with x']

# Scale the columns for training data and testing data
scaled_data  = pd.DataFrame( scaler.fit_transform(X[scale]), columns=scale, index=X[scale].index )

X = scaled_data

X

,x-box horizontal position of box,y-box vertical position of box,width of box,height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,-1.057698,0.291877,-1.053277,-0.164704,-1.144013,0.544130,2.365097,-1.714360,0.344994,-0.917071,1.347774,0.034125,-1.305948,-0.219082,-1.438153,0.122911
1,0.510385,1.502358,-1.053277,0.719730,-0.687476,1.531305,-1.075326,0.137561,-0.495072,1.895968,-1.312807,0.514764,-0.448492,-0.219082,0.120081,1.359441
2,-0.012309,1.199738,0.435910,1.161947,1.138672,1.531305,-0.645273,-0.973591,0.344994,0.690380,-1.312807,-0.446513,-0.019764,-0.865626,-0.269477,0.741176
3,1.555774,1.199738,0.435910,0.277513,-0.230939,-0.936631,0.644886,-0.232823,0.344994,-1.720796,-0.932724,0.995402,1.266419,1.074008,-0.659036,0.122911
4,-1.057698,-1.826464,-1.053277,-1.933571,-1.144013,0.544130,-0.645273,0.507945,0.344994,-0.917071,-0.552641,0.514764,-0.877220,-0.865626,0.509640,1.359441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-1.057698,-1.523844,-1.053277,-1.049137,-0.687476,0.050543,-0.215220,0.878329,0.344994,-0.917071,-0.172558,-1.888428,-0.448492,-0.219082,-0.269477,-0.495354
19996,1.555774,0.897117,1.428701,1.161947,0.225598,-1.430218,0.214833,0.507945,1.605094,1.494105,0.967691,2.437316,-0.448492,0.427463,-0.269477,-0.495354
19997,1.033079,0.594497,0.435910,0.719730,0.682135,-0.443044,1.504991,-0.603207,0.765028,1.092242,0.967691,-1.407789,-0.448492,2.367097,-0.659036,-2.350149
19998,-1.057698,-1.221224,-0.556881,-1.491354,-1.144013,0.544130,-0.215220,-0.973591,0.344994,0.690380,-0.172558,0.034125,-0.877220,0.427463,0.509640,0.122911


In [15]:
# Initialize lists to store performance scores
train_acc_scores = []
test_acc_scores = []
train_roc_scores = []
test_roc_scores = []
train_f1_scores = []
test_f1_scores = []

counter =  1 # for random state

for i in range(TRIALS):
    # Gets 5000 random samples
    X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=5000)
    
    counter += 1
    
    # Create grid search
    clf = GridSearchCV(rf, param, cv=StratifiedKFold(n_splits=5),
                   scoring=['accuracy','roc_auc','f1'], refit=False, verbose=2, n_jobs=-1)
    
    # Fit grid search
    best_model = clf.fit(X_train, y_train)
    
    # Best accuracy
    acc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ]
    # Best roc_auc
    roc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc']) ]
    # Best f1
    f1 = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1']) ]
    
    # Model Training
    # Best accuracy model, Best roc auc model, Best f1 model
    acc_model, roc_model, f1_model = best_models(acc, roc, f1)

    # Model fitting on training data
    acc_model.fit(X_train, y_train)
    roc_model.fit(X_train, y_train)
    f1_model.fit(X_train, y_train)
    
    # Model predicting
    # Accuracy model
    acc_train_pred = acc_model.predict(X_train)
    acc_test_pred = acc_model.predict(X_test)

    # Roc Auc model
    roc_train_pred = roc_model.predict_proba(X_train)[:,1]
    roc_test_pred = roc_model.predict_proba(X_test)[:,1]

    # f1 model
    f1_train_pred = f1_model.predict(X_train)
    f1_test_pred = f1_model.predict(X_test)
    
    # Appending score performances to train/test array
    # Best accuracy model scores
    train_acc_scores.append( accuracy_score(y_train,acc_train_pred) )
    test_acc_scores.append( accuracy_score(y_test,acc_test_pred) )
    
    # Best roc auc model scores
    train_roc_scores.append( roc_auc_score(y_train,roc_train_pred) )
    test_roc_scores.append( roc_auc_score(y_test,roc_test_pred) )
    
    # Best f1 model scores
    train_f1_scores.append( f1_score(y_train,f1_train_pred) )
    test_f1_scores.append( f1_score(y_test,acc_test_pred) )

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   24.9s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   25.1s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   26.8s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   27.3s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   26.3s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   26.6s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   28.0s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   28.3s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:   26.2s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   26.3s finished


In [16]:
# Saves scores to a DataFrame and prints
RF_LETTERO_TRAIN = pd.DataFrame([train_acc_scores, train_roc_scores, train_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_LETTERO_TRAIN.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_LETTERO_TRAIN,'\n')

RF_LETTERO_TEST = pd.DataFrame([test_acc_scores, test_roc_scores, test_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_LETTERO_TEST.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_LETTERO_TEST)

     TRIAL 1  TRIAL 2  TRIAL 3  TRIAL 4  TRIAL 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0 

      TRIAL 1   TRIAL 2   TRIAL 3   TRIAL 4   TRIAL 5
ACC  0.985400  0.988667  0.990200  0.988467  0.989400
ROC  0.996582  0.996991  0.997103  0.997916  0.996722
FSC  0.781219  0.837476  0.855172  0.824721  0.846079


# *Cover Type Dataset*

In [17]:
# Import Cover Type Dataset
cov = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', header=None)

# Loops through the Series changing 0 to -1
for index, value in cov[54].iteritems():
    if value == 2:
        cov.at[index, 54] = 1
    else:
        cov.at[index, 54] = -1

cov

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,-1
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,-1
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,-1
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,-1
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,-1
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,-1


In [18]:
# Data Preprocessing

# Separate x-values from target values
X = cov.drop([54], axis=1)
Y = cov[54] 

# Scaler
scaler = StandardScaler()

# Columns to scale
scale = cov.iloc[:,0:10]

# Scale the columns for training data and testing data
scaled_data  = pd.DataFrame( scaler.fit_transform(scale) )

for index in range(10):
    X[index] = scaled_data[index]

X

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,...,0,0,0,0,0,0,0,0,0,0
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,...,0,0,0,0,0,0,0,0,0,0
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,...,0,0,0,0,0,0,0,0,0,0
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,...,0,0,0,0,0,0,0,0,0,0
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,-2.012130,-0.023740,0.787408,-0.867697,-0.504653,-1.437962,1.040496,0.692069,-0.640851,-0.863386,...,0,0,0,0,0,0,0,0,0,0
581008,-2.029988,-0.032675,0.653865,-0.952383,-0.590424,-1.446299,1.040496,0.692069,-0.614724,-0.857345,...,0,0,0,0,0,0,0,0,0,0
581009,-2.047847,0.029873,0.386780,-0.985317,-0.676194,-1.449506,0.891075,0.894409,-0.327327,-0.850548,...,0,0,0,0,0,0,0,0,0,0
581010,-2.054990,0.128163,0.119694,-0.985317,-0.710502,-1.449506,0.666942,1.096749,0.012325,-0.842997,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Initialize lists to store performance scores
train_acc_scores = []
test_acc_scores = []
train_roc_scores = []
test_roc_scores = []
train_f1_scores = []
test_f1_scores = []

counter =  1 # for random state

for i in range(TRIALS):
    # Gets 5000 random samples
    X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=5000)
    
    counter += 1
    
    # Create grid search
    clf = GridSearchCV(rf, param, cv=StratifiedKFold(n_splits=5),
                   scoring=['accuracy','roc_auc','f1'], refit=False, verbose=2, n_jobs=-1)
    
    # Fit grid search
    best_model = clf.fit(X_train, y_train)
    
    # Best accuracy
    acc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_accuracy']) ]
    # Best roc_auc
    roc = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_roc_auc']) ]
    # Best f1
    f1 = best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1']) ]
    
    # Model Training
    # Best accuracy model, Best roc auc model, Best f1 model
    acc_model, roc_model, f1_model = best_models(acc, roc, f1)

    # Model fitting on training data
    acc_model.fit(X_train, y_train)
    roc_model.fit(X_train, y_train)
    f1_model.fit(X_train, y_train)
    
    # Model predicting
    # Accuracy model
    acc_train_pred = acc_model.predict(X_train)
    acc_test_pred = acc_model.predict(X_test)

    # Roc Auc model
    roc_train_pred = roc_model.predict_proba(X_train)[:,1]
    roc_test_pred = roc_model.predict_proba(X_test)[:,1]

    # f1 model
    f1_train_pred = f1_model.predict(X_train)
    f1_test_pred = f1_model.predict(X_test)
    
    # Appending score performances to train/test array
    # Best accuracy model scores
    train_acc_scores.append( accuracy_score(y_train,acc_train_pred) )
    test_acc_scores.append( accuracy_score(y_test,acc_test_pred) )
    
    # Best roc auc model scores
    train_roc_scores.append( roc_auc_score(y_train,roc_train_pred) )
    test_roc_scores.append( roc_auc_score(y_test,roc_test_pred) )
    
    # Best f1 model scores
    train_f1_scores.append( f1_score(y_train,f1_train_pred) )
    test_f1_scores.append( f1_score(y_test,acc_test_pred) )

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.1min remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.0min remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.0min remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.1min remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done  38 out of  40 | elapsed:  1.0min remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


In [20]:
# Saves scores to a DataFrame and prints
RF_COV_TRAIN = pd.DataFrame([train_acc_scores, train_roc_scores, train_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_COV_TRAIN.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_COV_TRAIN,'\n')

RF_COV_TEST = pd.DataFrame([test_acc_scores, test_roc_scores, test_f1_scores],
                    columns=['TRIAL 1','TRIAL 2','TRIAL 3','TRIAL 4','TRIAL 5'])
RF_COV_TEST.rename(index={0:'ACC', 1:'ROC', 2:'FSC'}, inplace=True)
print(RF_COV_TEST)


     TRIAL 1  TRIAL 2  TRIAL 3  TRIAL 4  TRIAL 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0 

      TRIAL 1   TRIAL 2   TRIAL 3   TRIAL 4   TRIAL 5
ACC  0.823861  0.826809  0.826330  0.820420  0.820247
ROC  0.901971  0.904860  0.903450  0.903520  0.899825
FSC  0.822566  0.826916  0.823174  0.817224  0.820058


# Mean Scores

In [21]:
# Returns the average rounded to three decimal places
def average_scores(list1, list2, list3, list4, index):
    # index = 0, 1, 2 for accuracy, roc auc, and f1, respectively
    average = round(mean(list1[index]+list2[index]+list3[index]+list4[index]), 3)
    return average

def average_by_metric(list):
    average = round(mean(list[0]+list[1]+list[2]), 3)
    return average

# Get training scores from DataFrame
RF_ADULT_TRAIN_LIST = RF_ADULT_TRAIN.values.tolist()
RF_LETTERAM_TRAIN_LIST = RF_LETTERAM_TRAIN.values.tolist()
RF_LETTERO_TRAIN_LIST = RF_LETTERO_TRAIN.values.tolist()
RF_COV_TRAIN_LIST = RF_COV_TRAIN.values.tolist()

# Get testing scores from DataFrame
RF_ADULT_TEST_LIST = RF_ADULT_TEST.values.tolist()
RF_LETTERAM_TEST_LIST = RF_LETTERAM_TEST.values.tolist()
RF_LETTERO_TEST_LIST = RF_LETTERO_TEST.values.tolist()
RF_COV_TEST_LIST = RF_COV_TEST.values.tolist()

In [22]:
print('Mean RF training accuracy:', average_scores(RF_ADULT_TRAIN_LIST,RF_LETTERAM_TRAIN_LIST,
                                                     RF_LETTERO_TRAIN_LIST,RF_COV_TRAIN_LIST,0))
print('Mean RF training roc auc:', average_scores(RF_ADULT_TRAIN_LIST,RF_LETTERAM_TRAIN_LIST,
                                                     RF_LETTERO_TRAIN_LIST,RF_COV_TRAIN_LIST,1))
print('Mean RF training f1:', average_scores(RF_ADULT_TRAIN_LIST,RF_LETTERAM_TRAIN_LIST,
                                                     RF_LETTERO_TRAIN_LIST,RF_COV_TRAIN_LIST,2))

Mean RF training accuracy: 1.0
Mean RF training roc auc: 1.0
Mean RF training f1: 1.0


In [23]:
print('Mean RF testing accuracy:', average_scores(RF_ADULT_TEST_LIST,RF_LETTERAM_TEST_LIST,
                                                     RF_LETTERO_TEST_LIST,RF_COV_TEST_LIST,0))
print('Mean RF testing roc auc:', average_scores(RF_ADULT_TEST_LIST,RF_LETTERAM_TEST_LIST,
                                                     RF_LETTERO_TEST_LIST,RF_COV_TEST_LIST,1))
print('Mean RF testing f1:', average_scores(RF_ADULT_TEST_LIST,RF_LETTERAM_TEST_LIST,
                                                     RF_LETTERO_TEST_LIST,RF_COV_TEST_LIST,2))

Mean RF testing accuracy: 0.902
Mean RF testing roc auc: 0.948
Mean RF testing f1: 0.815


In [24]:
print('Mean ADULT testing performance by metric:', average_by_metric(RF_ADULT_TEST_LIST))
print('Mean LetterAMPos testing performance by metric:', average_by_metric(RF_LETTERAM_TEST_LIST))
print('Mean LetterOPos testing performance by metric:', average_by_metric(RF_LETTERO_TEST_LIST))
print('Mean COV testing performance by metric:', average_by_metric(RF_COV_TEST_LIST))

Mean ADULT testing performance by metric: 0.806
Mean LetterAMPos testing performance by metric: 0.961
Mean LetterOPos testing performance by metric: 0.938
Mean COV testing performance by metric: 0.849
